In [33]:
import logging,qdrant_client
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,Settings,StorageContext
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.qdrant import QdrantVectorStore


In [34]:
# load documents
documents = SimpleDirectoryReader("./doc").load_data()

In [35]:
Settings.llm=Ollama(model="llama3",request_timeout=120.0)

In [36]:
Settings.embed_model=OllamaEmbedding(model_name="snowflake-arctic-embed")

In [37]:
Settings.text_splitter=SentenceSplitter(chunk_size=1024,chunk_overlap=20)

In [38]:
client=qdrant_client.QdrantClient(location=":memory:")
vector_store=QdrantVectorStore(client=client,collection_name="sampledata")

In [39]:
storage_context=StorageContext.from_defaults(vector_store=vector_store)

In [40]:
index = VectorStoreIndex.from_documents(documents=documents,storage_context=storage_context)

In [41]:
index.storage_context.persist(persist_dir="db")

In [47]:
query_engine=index.as_query_engine(response_mode="refine",verbose=True,similarity_top_k=10)

In [87]:
query_str = "code to add two numbers"

In [88]:
system_prompt = (
   "You are an AI assistant specialized in providing information from the uploaded document. "
   "Please ensure that your responses are strictly derived from the content of the document. "
   "If the information is not found in the document, please indicate that explicitly."
)  

In [89]:
query_with_prompt=f"{system_prompt}\nUser query:{query_str}"

In [90]:
query_engine = index.as_query_engine()
response = query_engine.query(query_with_prompt)
display(Markdown(f"{response}"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


I'm not finding any information related to adding two numbers in the provided context. The uploaded document appears to be about Customer Identification and KYC norms, and it does not contain any mathematical operations or coding examples. Therefore, I must indicate that the requested code is not found in the document. If you're looking for help with a specific math problem or coding task, feel free to ask, and I'll do my best to assist you within the context of this uploaded document!

In [91]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_with_prompt)
display(Markdown(f"{response}"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


I'm happy to help! However, I must clarify that the provided context information does not contain any relevant information about adding two numbers or performing mathematical operations. The context appears to be focused on customer identification and Know Your Customer (KYC) norms in banking and financial services.

Since the query "code to add two numbers" is unrelated to the provided context, I will explicitly indicate that the answer cannot be found within the document.

In [92]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


In [81]:
hyde_doc

"The humble printed cheque form has been a stalwart in the world of financial transactions for centuries. Despite the rise of digital payment methods, printed cheques continue to offer several key merits that make them an attractive option for many individuals and businesses.\n\nFirstly, printed cheques provide a tangible record of a transaction, allowing both the payer and payee to verify the details of the payment. This physical evidence can be invaluable in situations where disputes arise or when auditing financial records is necessary. Furthermore, printed cheques are resistant to hacking and cyber attacks, providing an added layer of security for sensitive transactions.\n\nAnother significant merit of printed cheque forms is their flexibility. Unlike digital payments, which may have limited transaction amounts or require specific bank accounts, printed cheques can be customized to suit individual needs. For instance, businesses may use printed cheques to make bulk payments or to p